In [8]:
#Pentru a putea incarca fisiere in colab. Voi alege mai jos LPTester.py.
from google.colab import files
uploaded = files.upload()

from operator import add
from LPTester import test_batch

Saving LPTester.py to LPTester.py


In [9]:
### Reprezentare - construcție
CONST = "const"
ATOM = "atom"
VAR = "var"
CALL = "call"
AND = "&"
OR = "|"
NEG = "~"
OPS = [AND, OR, NEG]

TYPE = FUNC = 0
ARGS = 1

# întoarce un termen constant, cu valoarea specificată.
def make_const(value):
    return (CONST, value)

# întoarce un termen care este o variabilă, cu numele specificat.
def make_var(name):
    return (VAR, name)

# întoarce un termen care este un apel al funcției specificate, pe restul argumentelor date.
# E.g. pentru a construi termenul add[1, 2, 3] vom apela
#  make_function_call(add, make_const(1), make_const(2), make_const(3))
# !! ATENȚIE: python dă args ca tuplu cu restul argumentelor, nu ca listă. Se poate converti la listă cu list(args)
def make_function_call(function, *args):
    return (CALL, (function, list(args)))

# întoarce o formulă formată dintr-un atom care este aplicarea predicatului dat pe restul argumentelor date.
# !! ATENȚIE: python dă args ca tuplu cu restul argumentelor, nu ca listă. Se poate converti la listă cu list(args)
def make_atom(predicate, *args):
    return (ATOM, (predicate, list(args)))

# întoarce o formulă care este negarea propoziției date.
# get_args(make_neg(s1)) va întoarce [s1]
def make_neg(sentence):
    return (NEG, [sentence])

# întoarce o formulă care este conjuncția propozițiilor date (2 sau mai multe).
# e.g. apelul make_and(s1, s2, s3, s4) va întoarce o structură care este conjuncția s1 ^ s2 ^ s3 ^ s4
#  și get_args pe această structură va întoarce [s1, s2, s3, s4]
def make_and(sentence1, sentence2, *others):
    return (AND, [sentence1, sentence2] + list(others))

# întoarce o formulă care este disjuncția propozițiilor date.
# e.g. apelul make_or(s1, s2, s3, s4) va întoarce o structură care este disjuncția s1 V s2 V s3 V s4
#  și get_args pe această structură va întoarce [s1, s2, s3, s4]
def make_or(sentence1, sentence2, *others):
    return (OR, [sentence1, sentence2] + list(others))

# întoarce o copie a formulei sau apelul de funcție date, în care argumentele au fost înlocuite
#  cu cele din lista new_args.
# e.g. pentru formula p(x, y), înlocuirea argumentelor cu lista [1, 2] va rezulta în formula p(1, 2).
# Noua listă de argumente trebuie să aibă aceeași lungime cu numărul de argumente inițial din formulă.
def replace_args(formula, new_args):
    if is_atom(formula) or is_function_call(formula):
        return (formula[TYPE], (formula[ARGS][TYPE], new_args))
    return (formula[TYPE], new_args)
    
    
### Reprezentare - verificare

# întoarce adevărat dacă f este un termen.
def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)

# întoarce adevărat dacă f este un termen constant.
def is_constant(f):
    return f[TYPE] == CONST

# întoarce adevărat dacă f este un termen ce este o variabilă.
def is_variable(f):
    return f[TYPE] == VAR

# întoarce adevărat dacă f este un apel de funcție.
def is_function_call(f):
    return f[TYPE] == CALL

# întoarce adevărat dacă f este un atom (aplicare a unui predicat).
def is_atom(f):
    return f[TYPE] == ATOM

# întoarce adevărat dacă f este o propoziție validă.
def is_sentence(f):
    return is_atom(f) or f[TYPE] in OPS

# întoarce adevărat dacă formula f este ceva ce are argumente.
def has_args(f):
    return is_function_call(f) or is_sentence(f)


### Reprezentare - verificare

# pentru constante (de verificat), se întoarce valoarea constantei; altfel, None.
def get_value(f):
    return f[ARGS] if is_constant(f) else None

# pentru variabile (de verificat), se întoarce numele variabilei; altfel, None.
def get_name(f):
    return f[ARGS] if is_variable(f) else None

# pentru apeluri de funcții, se întoarce funcția;
# pentru atomi, se întoarce numele predicatului; 
# pentru propoziții compuse, se întoarce un șir de caractere care reprezintă conectorul logic (e.g. ~, A sau V);
# altfel, None
def get_head(f):
    if is_atom(f) or is_function_call(f):
        return f[ARGS][FUNC]
    if is_sentence(f):
        return f[TYPE]

# pentru propoziții sau apeluri de funcții, se întoarce lista de argumente; altfel, None.
# Vezi și "Important:", mai sus.
def get_args(f):
    if is_atom(f) or is_function_call(f):
        return f[ARGS][ARGS]
    if is_sentence(f):
        return f[ARGS]


test_batch(0, globals())

>>> Test batch [0]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
Test 11: OK
Test 12: OK
Test 13: OK
Test 14: OK
Test 15: OK
Test 16: OK
Test 17: OK
Test 18: OK
Test 19: OK
Test 20: OK
Test 21: OK
Test 22: OK
Test 23: OK
Test 24: OK
Test 25: OK
Test 26: OK
Test 27: OK
Test 28: OK
Test 29: OK
Test 30: OK
Test 31: OK
Test 32: OK
>>>  33 / 33 tests successful.


In [10]:
# Afișează formula f. Dacă argumentul return_result este True, rezultatul nu este afișat la consolă, ci întors.
def print_formula(f, return_result = False):
    ret = ""
    if is_term(f):
        if is_constant(f):
            ret += str(get_value(f))
        elif is_variable(f):
            ret += "?" + get_name(f)
        elif is_function_call(f):
            ret += str(get_head(f)) + "[" + "".join([print_formula(arg, True) + "," for arg in get_args(f)])[:-1] + "]"
        else:
            ret += "???"
    elif is_atom(f):
        ret += str(get_head(f)) + "(" + "".join([print_formula(arg, True) + ", " for arg in get_args(f)])[:-2] + ")"
    elif is_sentence(f):
        # negation, conjunction or disjunction
        args = get_args(f)
        if len(args) == 1:
            ret += str(get_head(f)) + print_formula(args[0], True)
        else:
            ret += "(" + str(get_head(f)) + "".join([" " + print_formula(arg, True) for arg in get_args(f)]) + ")"
    else:
        ret += "???"
    if return_result:
        return ret
    print(ret)
    return
    
# Verificare construcție și afișare
# Ar trebui ca ieșirea să fie similară cu: (A (V ~P(?x) Q(?x)) T(?y, <built-in function add>[1,2]))
formula1 = make_and(
    make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))),
    make_atom("T", make_var("y"), make_function_call(add, make_const(1), make_const(2))))
print_formula(formula1)

(& (| ~P(?x) Q(?x)) T(?y, <built-in function add>[1,2]))


In [11]:
# Aplică în formula f toate elementele din substituția dată și întoarce formula rezultată
def substitute(f, substitution):
    if substitution is None:
        return None
    if is_variable(f) and (get_name(f) in substitution):
        return substitute(substitution[get_name(f)], substitution)
    if has_args(f):
        return replace_args(f, [substitute(arg, substitution) for arg in get_args(f)])
    return f

def test_formula(x, copyy = False):
    fun = make_function_call(add, make_const(1), make_const(2))
    return make_and(make_or(make_neg(make_atom("P", make_const(x))), make_atom("Q", make_const(x))), \
                    make_atom("T", fun if copyy else make_var("y"), fun))

# Test (trebuie să se vadă efectele substituțiilor în formulă)
test_batch(1, globals())

>>> Test batch [1]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
>>>  5 / 5 tests successful.


In [12]:
# Verifică dacă variabila v apare în termenul t, având în vedere substituția subst.
# Întoarce True dacă v apare în t (v NU poate fi înlocuită cu t), și False dacă v poate fi înlocuită cu t.
from functools import partial

def occur_check(v, t, subst):
    if v == t:
        return True
    if get_name(t) in subst:
        return occur_check(v, substitute(t, subst), subst)
    if is_function_call(t):
        return any(map(lambda arg: occur_check(v, arg, subst), get_args(t)))
    return False

# Test!
test_batch(2, globals())

>>> Test batch [2]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
>>>  7 / 7 tests successful.


In [13]:
# Unifică formulele f1 și f2, sub o substituție existentă subst.
# Rezultatul unificării este o substituție (dicționar nume-variabilă -> termen),
#  astfel încât dacă se aplică substituția celor două formule, rezultatul este identic.
def unify(f1, f2, subst = None):
    if not subst:
        subst = {}
    
    st = [(f1, f2)]

    while st:
        f1, f2 = st.pop()

        while get_name(f1) in subst:
            f1 = substitute(f1, subst)
        while get_name(f2) in subst: 
            f2 = substitute(f2, subst)

        if f1 == f2:
            continue

        if is_variable(f1):
            if occur_check(f1, f2, subst):
                return False

            subst[get_name(f1)] = f2
        elif is_variable(f2):
            if occur_check(f2, f1, subst):
                return False
 
            subst[get_name(f2)] = f1
        elif has_args(f1) and has_args(f2):
            if get_head(f1) == get_head(f2):
                f1_args = get_args(f1)
                f2_args = get_args(f2)

                if len(f1_args) == len(f2_args):
                    st += zip(f1_args, f2_args)
            else:
                return False
        else:
            return False

    return subst

# Test!
test_batch(3, globals())

>>> Test batch [3]
Test 0: OK, got < False >
Test 1: OK, got < {?x -> A} >
Test 2: OK, got < {?x -> A} >
Test 3: OK, got < {?x -> A} >
Test 4: OK, got < {?x -> A} >
Test 5: OK, got < {?x -> <built-in function add>[?Z,5]} >
Test 6: OK, got < {?z -> C, ?y -> B, ?x -> A} >
Test 7: OK, got < {?y -> 3, ?x -> 2} >
Test 8: OK, got < False >
Test 9: OK, got < False >
Test 10: OK, got < {} >
Test 11: OK, got < {?x -> ?y} >
Test 12: OK, got < {?y -> ?x} >
Test 13: OK, got < {?x -> A} >
Test 14: OK, got < False >
Test 15: OK, got < {?x -> A} >
Test 16: OK, got < {?x -> A, ?z -> C, ?y -> B} >
Test 17: OK, got < {?x -> A} >
Test 18: OK, got < {?x -> ?y} >
Test 19: OK, got < {?y -> ?x} >
Test 20: OK, got < {?z -> 5, ?y -> 5, ?x -> 5} >
Test 21: OK, got < {?z -> <built-in function add>[1,2], ?y -> 4, ?x -> 3} >
>>>  22 / 22 tests successful.


In [14]:
from copy import deepcopy
from functools import reduce

# în această celulă se găsesc câteva funcții utilizate intern

dummy = make_atom("P")
[and_name, or_name, neg_name] = [get_head(s) for s in [make_and(dummy, dummy), make_or(dummy, dummy), make_neg(dummy)]]
def pFail(message, f):
    print(message + " <" + str(f) + ">")
    return False
def check_term(T):
    if is_constant(T):
        return (get_value(T) is not None) or pFail("The value of the constant is None", T)
    if is_variable(T):
        return (get_name(T) is not None) or pFail("The name of the variable is None", T)
    if is_function_call(T):
        return not [t for t in get_args(T) if not check_term(t)] and \
            (get_head(T) is not None or pFail("Function is not callable", T))
    return pFail("Term is not one of constant, variable or function call", T)
def check_atom(A):
    if is_atom(A):
        return not [t for t in get_args(A) if not check_term(t)] and \
            (get_head(A) is not None or pFail("Predicate name is None", A))
    return pFail("Is not an atom", A)
def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [and_name, or_name]:
            return (len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S)) \
                and not [s for s in get_args(S) if not check_sentence(s)]
        if get_head(S) == neg_name:
            return (len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) \
                and check_sentence(get_args(S)[0])
    return pFail("Not sentence or unknown type", S)

def add_statement(kb, conclusion, *hypotheses):
    s = conclusion if not hypotheses else make_or(*([make_neg(s) for s in hypotheses] + [conclusion]))
    if check_sentence(s):
        kb.append(s)
        print("OK: Added statement " + print_formula(s, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(s, True)+"><" + str(s) + ">")
    return False

var_no = 0;

def assign_next_var_name():
    global var_no
    var_no += 1
    return "v" + str(var_no)

def gather_vars(S):
    return [get_name(S)] if is_variable(S) else \
        [] if not has_args(S) else reduce(lambda res, a: res + gather_vars(a), get_args(S), [])

def make_unique_var_names(KB):
    global var_no
    var_no = 0
    return [substitute(S, {var: make_var(assign_next_var_name()) for var in gather_vars(S)}) for S in KB]           
            
def print_KB(KB):
    print("KB now:")
    for s in KB:
        print("\t\t\t" + print_formula(s, True))

In [15]:
# KB 1
# based on an example in Artificial Intelligence - A Modern Approach
KB_America = []
#0 Mr West is a US general
add_statement(KB_America, make_atom("USGeneral", make_const("West")))
#1 General Awesome is also a US general
add_statement(KB_America, make_atom("USGeneral", make_const("General_Awesome")))
#2 General Awesome is Awesome
add_statement(KB_America, make_atom("Awesome", make_const("General_Awesome")))
#3 Nono is an enemy of America
add_statement(KB_America, make_atom("Enemy", make_const("Nono"), make_const("America")))
#4 M1 is a type of missile
add_statement(KB_America, make_atom("Missile", make_const("M1")))
#5 Nono has the M1 missile
add_statement(KB_America, make_atom("Owns", make_const("Nono"), make_const("M1")))

#6 any US general is an American
add_statement(KB_America, make_atom("American", make_var("x")), make_atom("USGeneral", make_var("x")))
#7 any missle is a weapon
add_statement(KB_America, make_atom("Weapon", make_var("x")), make_atom("Missile", make_var("x")))
#8 if anyone owns a missile, it is General West that sold them that missile
add_statement(KB_America, make_atom("Sells", make_const("West"), make_var("y"), make_var("x")), make_atom("Owns", make_var("x"), make_var("y")), make_atom("Missile", make_var("y")))
#9 any American who sells weapons to a hostile is a criminal
add_statement(KB_America, make_atom("Criminal", make_var("x")), make_atom("Weapon", make_var("y")), make_atom("Sells", make_var("x"), make_var("y"), make_var("z")), make_atom("Hostile", make_var("z")), make_atom("American", make_var("x")))
#10 any enemy of America is called a hostile
add_statement(KB_America, make_atom("Hostile", make_var("x")), make_atom("Enemy", make_var("x"), make_const("America")))
#11 America is awesome if at least an American is awesome
add_statement(KB_America, make_atom("Awesome", make_const("America")), make_atom("American", make_var("x")), make_atom("Awesome", make_var("x")))

KB_America = make_unique_var_names(KB_America)

print_KB(KB_America)

OK: Added statement USGeneral(West)
OK: Added statement USGeneral(General_Awesome)
OK: Added statement Awesome(General_Awesome)
OK: Added statement Enemy(Nono, America)
OK: Added statement Missile(M1)
OK: Added statement Owns(Nono, M1)
OK: Added statement (| ~USGeneral(?x) American(?x))
OK: Added statement (| ~Missile(?x) Weapon(?x))
OK: Added statement (| ~Owns(?x, ?y) ~Missile(?y) Sells(West, ?y, ?x))
OK: Added statement (| ~Weapon(?y) ~Sells(?x, ?y, ?z) ~Hostile(?z) ~American(?x) Criminal(?x))
OK: Added statement (| ~Enemy(?x, America) Hostile(?x))
OK: Added statement (| ~American(?x) ~Awesome(?x) Awesome(America))
KB now:
			USGeneral(West)
			USGeneral(General_Awesome)
			Awesome(General_Awesome)
			Enemy(Nono, America)
			Missile(M1)
			Owns(Nono, M1)
			(| ~USGeneral(?v2) American(?v2))
			(| ~Missile(?v4) Weapon(?v4))
			(| ~Owns(?v9, ?v8) ~Missile(?v8) Sells(West, ?v8, ?v9))
			(| ~Weapon(?v12) ~Sells(?v16, ?v12, ?v14) ~Hostile(?v14) ~American(?v16) Criminal(?v16))
			(| ~Enem

In [16]:
# KB 2
# din cursul de IA
KB_Faster = []

def the_greyhound():
    return make_const("Greg")

#0 horses are faster than dogs
add_statement(KB_Faster, make_atom("Faster", make_var("x"), make_var("y")), make_atom("Horse", make_var("x")), make_atom("Dog", make_var("y")))
#1 there is a greyhound that is faster than any rabbit
add_statement(KB_Faster, make_atom("Faster", make_function_call(the_greyhound), make_var("z")), make_atom("Rabbit", make_var("z")))
#2 Harry is a horse
add_statement(KB_Faster, make_atom("Horse", make_const("Harry")))
#3 Ralph is a rabbit
add_statement(KB_Faster, make_atom("Rabbit", make_const("Ralph")))
#4 Greg is a greyhound
add_statement(KB_Faster, make_atom("Greyhound", make_function_call(the_greyhound)))
#5 A greyhound is a dog
add_statement(KB_Faster, make_atom("Dog", make_var("y")), make_atom("Greyhound", make_var("y")))
#6 transitivity
add_statement(KB_Faster, make_atom("Faster", make_var("x"), make_var("z")),
              make_atom("Faster", make_var("x"), make_var("y")), make_atom("Faster", make_var("y"), make_var("z")))

KB_Faster = make_unique_var_names(KB_Faster)

print_KB(KB_Faster)

OK: Added statement (| ~Horse(?x) ~Dog(?y) Faster(?x, ?y))
OK: Added statement (| ~Rabbit(?z) Faster(<function the_greyhound at 0x7f3b9d7fe680>[], ?z))
OK: Added statement Horse(Harry)
OK: Added statement Rabbit(Ralph)
OK: Added statement Greyhound(<function the_greyhound at 0x7f3b9d7fe680>[])
OK: Added statement (| ~Greyhound(?y) Dog(?y))
OK: Added statement (| ~Faster(?x, ?y) ~Faster(?y, ?z) Faster(?x, ?z))
KB now:
			(| ~Horse(?v3) ~Dog(?v4) Faster(?v3, ?v4))
			(| ~Rabbit(?v6) Faster(<function the_greyhound at 0x7f3b9d7fe680>[], ?v6))
			Horse(Harry)
			Rabbit(Ralph)
			Greyhound(<function the_greyhound at 0x7f3b9d7fe680>[])
			(| ~Greyhound(?v8) Dog(?v8))
			(| ~Faster(?v13, ?v11) ~Faster(?v11, ?v14) Faster(?v13, ?v14))


In [17]:
KB_test = []
add_statement(KB_test, make_atom("Q", make_var("x")), make_atom("P", make_var("x")))
add_statement(KB_test, make_atom("P", make_const("A")))

KB_test = make_unique_var_names(KB_test)
print_KB(KB_test)

OK: Added statement (| ~P(?x) Q(?x))
OK: Added statement P(A)
KB now:
			(| ~P(?v2) Q(?v2))
			P(A)


In [18]:
def is_positive_literal(L):
    return is_atom(L)
def is_negative_literal(L):
    global neg_name
    return get_head(L) == neg_name and is_positive_literal(get_args(L)[0])
def is_literal(L):
    return is_positive_literal(L) or is_negative_literal(L)

def resolves(C1, C2):
    #print("testing " + print_formula(C1, True) + " and " + print_formula(C2, True))
 
    # întoarce un tuplu (literal-din-C1, literal-din-C2, substituție)
    # unde literal-din-C1 și literal-din-C2 unifică sub substituție
    return False

# Test!
test_batch(4, globals())

>>> Test batch [4]
Test 0 : Result should be a 3-tuple, was < False >; Test was resolves USGeneral(General_Awesome) with (| ~USGeneral(?v2) American(?v2))
Test 1 : Result should be a 3-tuple, was < False >; Test was resolves Owns(Nono, M1) with (| ~Owns(?v9, ?v8) ~Missile(?v8) Sells(West, ?v8, ?v9))
Test 2 : Result should be a 3-tuple, was < False >; Test was resolves (| ~Weapon(?v12) ~Sells(?v16, ?v12, ?v14) ~Hostile(?v14) ~American(?v16) Criminal(?v16)) with (| ~USGeneral(?v2) American(?v2))
Test 3 : Result should be a 3-tuple, was < False >; Test was resolves Enemy(Nono, America) with (| ~Enemy(?v18, America) Hostile(?v18))
Test 4: OK, got < False >
Test 5: OK, got < False >
Test 6: OK, got < False >
Test 7: OK, got < False >
Test 8 : Result should be a 3-tuple, was < False >; Test was resolves (| P(?x) Q(5)) with (| ~Q(5) R(?y))
Test 9: OK, got < False >
>>>  5 / 10 tests successful.


In [19]:
# prints a 5-tuple resolvent representation (see below)
def print_r(R):
    if R is None:
        print("no resolvent")
    else:
        print("resolvent: " + print_formula(R[2], True) + "/" + print_formula(R[3], True) \
              + " {" + ", ".join([(k + ": " + str(R[4][k])) for k in R[4]]) + "}" \
              + "\n\t\t in " + print_formula(R[0], True) + "\n\t\t and " + print_formula(R[1], True))

In [20]:
from copy import deepcopy

VOID_CLAUSE = "The void clause"

def new_clause(resolvent):
    C1, C2, L1, L2, s = resolvent

    C1_args = deepcopy([C1] if is_literal(C1) else get_args(C1))
    C2_args = deepcopy([C2] if is_literal(C2) else get_args(C2))

    if L1 in C1_args:
        C1_args.remove(L1)
    if L2 in C2_args:
        C2_args.remove(L2)

    all_args = list(map(lambda a: substitute(a, s), C1_args + C2_args))

    C_args = []
    for arg in all_args:
        if arg not in C_args:
            C_args.append(arg)

    if not C_args:
        return VOID_CLAUSE
    elif len(C_args) == 1:
        return C_args[0]
    else:
        return make_or(*C_args)

# Test!
test_batch(5, globals())

>>> Test batch [5]
Test 0: OK, got < The void clause >
Test 1: OK, got < Q(?y) >
Test 2: OK, got < ~Q(5) >
Test 3: OK
>>>  4 / 4 tests successful.


In [23]:
def solve_problem(hypotheses, conclusion):
    KB = hypotheses[:]
    KB = [make_neg(conclusion)] + KB # puteți adăuga și la sfârșit (în funcție de strategie)
    Effort = 50 # puteți crește efortul dacă este necesar
    
    while Effort > 0:
        Effort -= 1
      
        # Se aleg două clauze Clauza1 și Clauza2, care nu au mai fost alese anterior
        
        # Se calculează un rezolvent, ca tuplu (Clauza1, Clauza2, Literal-din-clauza1, Literal-din-clauza2, substituție)
        resolvent = None # TODO

        print_r(resolvent)
        if resolvent is None:
            print("Failed. No resolving clauses. Effort left " + str(Effort))
            return False
        
        # Se calculează noua clauză de adăugat și se adaugă la baza de cunoștințe
        
        C = new_clause(resolvent)
        
        if C == VOID_CLAUSE:
            print("Done (effort left " + str(Effort) + ")")
            return True
        
        # update KB
        print("Added: " + print_formula(C, True))
        KB = [C] + KB

        print_KB(KB)
    print("Failed. Effort exhausted.")
                
        
#print_KB(KB_test)
solve_problem(deepcopy(KB_test), make_atom("Q", make_const("A")))
print("==========================================")

#print_KB(KB_America)
#solve_problem(deepcopy(KB_America), make_atom("Criminal", make_const("West")))
print("==========================================")

#print_KB(KB_Faster)
#solve_problem(deepcopy(KB_Faster), make_atom("Faster", make_const("Harry"), make_const("Ralph")))
print("==========================================")

no resolvent
Failed. No resolving clauses. Effort left 49
